- # 从get_result文件导入已经写好的读取kv值的方法
- # 导入要解析的短信文本数据

In [ ]:
import pandas as pd
from get_result import *
sms_data = pd.read_csv('data/after_filter.csv')

# 先调用之前的正则表达式文件先进行匹配，如果能够匹配到短句的信息就直接跳过，增加i,处理下一个句子

In [7]:
i = 0
raw_sentence = sms_data.loc[i]['原句']
print('-'*50+'短句'+'-'*50+'\n%s\n'%(sms_data.loc[i]['短句'])+'-'*100+'\n长句：%s'%(raw_sentence))
result = get_result(raw_sentence, kre_path='data/kre_914.csv')
print('='*100)
for k,v in result.items():
    print('{} : {}'.format(k, v))

--------------------------------------------------短句--------------------------------------------------
月手机账单：月日至月日，优惠后个人实际消费.元，合计.元。查询结果仅供参考，具体以出账结果为准。查明细回复至或登录手机营业厅http://dx..cn/gjdxsy 【中国移动】
----------------------------------------------------------------------------------------------------
长句：03月手机账单：03月1日至03月05日，优惠后个人实际消费2.96元，合计2.96元。查询结果仅供参考，具体以出账结果为准。查明细回复18至10086或登录手机营业厅http://dx.10086.cn/gjdxsy 【中国移动】
消费金额 : 2.96元
截止时间 : 03月05日


## 对将要写入的正则表达式的cn_key进行一个判断，以防不在规定的cn_key集合内
## 显示当前文件中cn_key对应的key的命名，以免key命名冲突

In [10]:
cn_key_set = ['可兑换积分','截止时间','缴费金额','缴费时间','话费余额','缴费渠道','充值号码',\
              '被充值号码','订单号','流水号','消费金额','账单月份','账户余额','备注']
cn_key  = '账单月份'
print(cn_key in cn_key_set)
kre = pd.read_csv('data/kre_914.csv')
for i in kre.index:
    if cn_key == kre.loc[i]['cn_key']:
        print(kre.loc[i]['key'])

True


In [11]:
import re
key = 'sqd_bill_month'
re_express = '(?P<{}>\d+.*?月)手机账单'.format(key)
find =re.search(re_express, raw_sentence)
if find:
    print(find)
    print('----------------------------------------------\n\
        {} : {}\n -----------------------------------------------'.format(key, find.group(key)))
else:
    print('not find')


<_sre.SRE_Match object; span=(0, 7), match='03月手机账单'>
----------------------------------------------
        sqd_bill_month : 03月
 -----------------------------------------------


# 把内容保存到kre数据中，其中last_commit是该句正则表达式所影响的句子，为以后改动正则表达式测试用

In [14]:
kre.loc[kre.index[-1]+1] =  {'key':key,
        'cn_key':cn_key,
        're':re_express,
        'last_commit':'原文ID:{}'.format(sms_data.loc[i]['原文ID'])}

In [15]:
kre

,cn_key,key,last_commit,re
0,截止时间,sqd_deadtime,[49],"截[至止].*?(?P<sqd_deadtime>(\d{4}年)?(\d{1,2}月)?(..."
1,话费余额,sqd_call_balance,NaN,话费余额[：为是]?(?P<sqd_call_balance>.*?[元块￥])
2,话费余额,sqd_family_call_balance,NaN,家庭帐户.*余额[:为是](?P<sqd_family_call_balance>-?\d+...
3,消费金额,sqd_family_call_cost,NaN,家庭帐户.*消费[:为是]?(?P<sqd_family_call_cost>-?\d+\....
4,话费余额,sqd_call_balance2,[30],总余额.*?(?P<sqd_call_balance2>\d+.*?[元块￥])
5,话费余额,sqd_call_balance1,[40],可用余额[为是]?：?(?P<sqd_call_balance1>-?\d+.*?[元块￥])
6,账户余额,sqd_account_balance,[67],账户余额[：为是]?(?P<sqd_account_balance>\d+.*?[元块￥])
7,话费余额,ldd_charge_balance,NaN,(话费)?余额还?剩?余?(为|是)?(:|：)?(?P<ldd_charge_balanc...
8,消费金额,ldd_charge_amount,NaN,(消费|产生话费)(:|：)?(?P<ldd_charge_amount>[\d.]+[元角...
9,缴费渠道,ldd_payment_channel,NaN,"通过(?P<ldd_payment_channel>.*?)(及时)?[冲缴交充]值.{0,..."


In [16]:
kre.to_csv('data/kre_914.csv', index=False)